# ROBOTIC

Using stable_baselines3 for robotic use case


### pip install

Installing gym and panda-gym.

In [6]:
!pip install gym
!pip install panda-gym
!pip install stable_baselines3
!pip install tensorboard

Processing /home/david_ekchajzer/.cache/pip/wheels/b3/6c/4b/cf1a6c9b28148410d3edb448f2e680c262da3ecca6ceaf4fd0/panda_gym-2.0.1-py3-none-any.whl


### import

Importing gym and panda-gym.

In [7]:
import gym
import panda_gym
import stable_baselines3
from stable_baselines3.common.logger import configure
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3 import HerReplayBuffer, DDPG

## Testing the environnment

In [8]:
env = gym.make('PandaPush-v2', render=True)

obs = env.reset()
done = False
while not done:
    action = env.action_space.sample() # random action
    obs, reward, done, info = env.step(action)

env.close()

pybullet build time: Apr 26 2022 03:12:14


### Setting up model
Hyper-parameter from https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/her.yml

In [3]:
env = gym.make("PandaPush-v2")
model = DDPG('MultiInputPolicy', env, replay_buffer_class=HerReplayBuffer, replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy='future',
        online_sampling=True,
    ), buffer_size = 1000000, tau = 0.05, learning_rate = 1e-3, verbose=1, batch_size = 2048, gamma = 0.95, policy_kwargs = dict(n_critics=2, net_arch=[512, 512, 512]))

model.save("PandaPush-v2-DDPG")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


### Training the model

In [ ]:
tmp_path = "/tmp/sb3_log/"
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])

env = gym.make("PandaPush-v2")
model = DDPG('MultiInputPolicy', env, replay_buffer_class=HerReplayBuffer, replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy='future',
        online_sampling=True,
    ), buffer_size = 1000000, tau = 0.05, learning_rate = 1e-3, verbose=1, batch_size = 2048, gamma = 0.95, policy_kwargs = dict(n_critics=2, net_arch=[512, 512, 512]))


model.set_parameters("PandaPush-v2-DDPG")

model.set_logger(new_logger)

checkpoint_callback = CheckpointCallback(save_freq=125_000, 
                                         save_path='.', 
                                         name_prefix='PandaPush-v2')

eval_callback = EvalCallback(env, 
                             best_model_save_path='eval_save', 
                             eval_freq=12_500)

callback_list = CallbackList([checkpoint_callback, eval_callback])

model.learn(total_timesteps=int(1e7), 
            callback=callback_list, 
            log_interval=100, 
            tb_log_name='logs_robotics_PandaPush')

2022-05-03 09:50:21.355032: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/david_ekchajzer/.local/lib/python3.8/site-packages/cv2/../../lib64::/home/david_ekchajzer/.mujoco/mujoco210/bin
2022-05-03 09:50:21.355175: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Logging to /tmp/sb3_log/
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


### Saving and cleaning the environnment

In [2]:
model.save("PandaPush-v2-DDPG")

del model
del env

NameError: name 'model' is not defined

### Testing the environnment

In [ ]:
import gym
import panda_gym
from stable_baselines3 import DDPG
from stable_baselines3.common.logger import configure

env = gym.make("PandaPush-v2", render=True)
model = DDPG.load("PandaPush-v2-DDPG", env=env)
obs = env.reset()
dones = False

while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

### installing sb3-contrib

In [1]:
!pip install sb3-contrib

     |████████████████████████████████| 62 kB 1.5 MB/s eta 0:00:011


### Testing community trained model

In [1]:
import gym
import panda_gym
from sb3_contrib import TQC
from stable_baselines3.common.logger import configure

env = gym.make("PandaPush-v2", render=True)
model = TQC.load("FetchPush-v1-rl-trained-agents", env=env)
obs = env.reset()
dones = False

while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

DeprecatedEnv: Env PandaPush-v1 not found (valid versions include ['PandaPush-v2'])